In [6]:
import requests
from bs4 import BeautifulSoup

url = 'https://weather.com/weather/hourbyhour/l/5cf1c39bb2bf5c7e97bb1192db1c9652535612f3f1b1fc2d4c87731357a3f7f0'
contenido_html = requests.get(url).text
soup = BeautifulSoup(contenido_html, 'lxml')


In [12]:
#Vemos todos los tags h2 con la class DetailsSummary--, que son los que contienen las horas:
#print(soup.find_all('h2',class_='DetailsSummary--daypartName--2FBp2'))
len(soup.find_all('h2',class_='DetailsSummary--daypartName--2FBp2'))

48

In [17]:
#importamos la libreria para regex, la cual vamos a necesitar para buscar clases 
# o id por palabras que comiencen de determinada manera

import re 

'''Lo primero que hacemos es crear un diccionario en el cual separemos los diferentes días
en los que tenemos el pronóstico del tiempo. Lo que nos interesa el clima por horas, pero
para cada día, y el tema es que la info está contenida toda junta en un mismo bloque.'''

dias_diccionario = {}

for dia in soup.find_all('h3', class_='HourlyForecast--longDate--1tdaJ'):
    key= dia.get('id').split('currentDateId')[1]
    value = dia.text
    dias_diccionario[key]= value

''' Lo que hicimos fue, crear un bucle para que recorra los 3 elementos encontrados.
Para cada elemento crea una key en donde lo que hace es tomar el texto del id, dividirlo
en 2 partes, la primera es la palabra común a todos los id "currenDateId" y la segunda parte 
que queda, es el número de Id, propiamente dicho, que es el que queremos utilizar para 
identificar a cada día de la semana que se está mostrando con las temperaturas en los 
horarios correspondientes. Al tomar la segunda parte de split, nos quedamos con el número
como Id. 
En segundo lugar, creamos el valor que corresponde a la key creada anteriormente, que va a 
ser igual al texto dentro del tag h3, con el nombre del día.
Por último agregamos al diccionario en blanco que creamos al ppio. el key y value de cada día.
Como Resultado, obtenemos un diccionario, con 3 keys y sus valores asociados.
'''
dias_diccionario

{'0': 'Thursday, November 18',
 '23': 'Friday, November 19',
 '47': 'Saturday, November 20'}

In [26]:
''' Hay otra forma de hacerlo, tomando el id, en lugar de la clase, porque lo que puede pasar
es que el nombre de las clases vayan variando a futuro, por ende, si queremos asegurarnos, 
podemos tomar un atributo más estático, como el ID, y trabajar con él, en vez de realizar 
la búsqueda por clase.'''

dias_diccionario = {}

for dia in soup.find_all('h3', attrs={'id':re.compile("^currentDateId")}): #se lee los id que empiecen con la palabra "currentId"
    key = dia.get('id').split("currentDateId")[1]
    value = dia.text
    dias_diccionario[key]=value 


''' attrs={'id':re.compile("^currentDateId")}): se lee todos los id que empiecen 
con "currentDateId". Si no se hiciera de esta forma, y se buscara el ID con el nombre 
que aparece en el momento en que hago la búsqueda, estaría poniendo un nombre fijo, 
por ejemplo currentDateId11. El problema es que el ID se llama así ese día, pero luego
varía, entonces si se corre el código al día siguiente, no va a encontrar nada, porque
tal vez, al otro día, se llame currentDateId35. Por eso conviene hacer la búsqueda
siempre por la palabra que compartan todos los id de los elementos, y ese número que
es variable separarlo con un split en dos palabras, y luego tomar ese pedacito de palabra
que contiene el número que lo identifica. Es muy bueno lo que hace, no la tenía esa!
'''
dias_diccionario

{'0': 'Thursday, November 18',
 '23': 'Friday, November 19',
 '47': 'Saturday, November 20'}

In [57]:
datos = {
    "dia" : [],
    "hora": [],
    "temperatura": [],
    "nubosidad": [], 
    "precipitacion": [], 
    "viento": []

}
#En este diccionario, cada key va a tener una lista de valores, en el anterior hacía el for
# para los 3 valores expecíficos de los días.

for datos_por_hora in soup.find_all('summary', class_="Disclosure--Summary--UuybP DaypartDetails--Summary--3IBUr Disclosure--hideBorderOnSummaryOpen--ZdSDc"):
    key = datos_por_hora.get('aria-labelledby').split()[0].split('currentDateId')[1]
    datos['dia'].append(dias_diccionario[key])
    datos['hora'].append(datos_por_hora.find('h2', attrs={'data-testid':"daypartName"}).text)
    datos['temperatura'].append(datos_por_hora.find('div', attrs={'data-testid':"detailsTemperature"}).span.text)
    datos['nubosidad'].append(datos_por_hora.find('div',attrs={'data-testid':'wxIcon'}).span.text)
    datos['precipitacion'].append(datos_por_hora.find('div',attrs={'data-testid':'Precip'}).span.text)
    datos['viento'].append(datos_por_hora.find('div',attrs={'data-testid':'wind'}).span.text)

In [58]:
datos

{'dia': ['Thursday, November 18',
  'Thursday, November 18',
  'Thursday, November 18',
  'Thursday, November 18',
  'Thursday, November 18',
  'Thursday, November 18',
  'Thursday, November 18',
  'Thursday, November 18',
  'Thursday, November 18',
  'Thursday, November 18',
  'Thursday, November 18',
  'Thursday, November 18',
  'Thursday, November 18',
  'Thursday, November 18',
  'Thursday, November 18',
  'Thursday, November 18',
  'Thursday, November 18',
  'Thursday, November 18',
  'Thursday, November 18',
  'Thursday, November 18',
  'Thursday, November 18',
  'Thursday, November 18',
  'Thursday, November 18',
  'Friday, November 19',
  'Friday, November 19',
  'Friday, November 19',
  'Friday, November 19',
  'Friday, November 19',
  'Friday, November 19',
  'Friday, November 19',
  'Friday, November 19',
  'Friday, November 19',
  'Friday, November 19',
  'Friday, November 19',
  'Friday, November 19',
  'Friday, November 19',
  'Friday, November 19',
  'Friday, November 19

In [59]:
import pandas as pd

df = pd.DataFrame(datos)
df

,dia,hora,temperatura,nubosidad,precipitacion,viento
0,"Thursday, November 18",1 am,60°,Mostly Clear,1%,NNW 5 mph
1,"Thursday, November 18",2 am,59°,Mostly Clear,1%,NNW 6 mph
2,"Thursday, November 18",3 am,58°,Clear,1%,NNW 5 mph
3,"Thursday, November 18",4 am,56°,Clear,1%,NW 5 mph
4,"Thursday, November 18",5 am,55°,Clear,1%,NW 5 mph
5,"Thursday, November 18",6 am,55°,Sunny,1%,NW 5 mph
6,"Thursday, November 18",7 am,58°,Sunny,1%,NW 5 mph
7,"Thursday, November 18",8 am,62°,Sunny,0%,NW 6 mph
8,"Thursday, November 18",9 am,65°,Sunny,0%,NW 6 mph
9,"Thursday, November 18",10 am,68°,Sunny,0%,NNW 6 mph
